<a href="https://colab.research.google.com/github/danielsoy/ALOCC-CVPR2018/blob/master/unsupervised-anomaly-detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unsupervised anomaly detection using Anomalib library

_________________________
Author: Dennis Hernando NÚÑEZ FERNÁNDEZ    
Author website: [https://dennishnf.com](https://dennishnf.com)   

Modified by: Aditya Bhattacharya

Collaborator website: [https://aditya-bhattacharya.net/](https://aditya-bhattacharya.net/)    
_________________________    

## Pre-installation settings

All the experiments were runned on a machine with Ubuntu 20.04 LTS and NVIDIA GeForce GTX 1050 Ti with 4GB.    
In addition, this notebook was locally implemented using Conda and in a virtual environment.     
Therefore, to create a virtual enviroment in Conda, the following commands were introduced in the terminal:    

```
$ yes | conda create -n anomalib_env python=3.8
$ conda activate anomalib_env
```

## Installing AnomaLib

In [1]:
!pip install matplotlib==3.1.3 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 MB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for matplotlib: filename=matplotlib-3.1.3-cp39-cp39-linux_x86_64.whl size=12062530 sha256=efb90391eee645a128b5b71aa3cfc4c6f9c01989fb101ca30b21ce8679cbce33
  Stored in directory: /root/.cache/pip/wheels/88/5f/33/d7b8943eba74fdfbd535c83cefcf366c25b0f9cb6424e763e7
Successfully built matplotlib
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.1
    Uninstalling matplotlib-3.7.1:
      Successfully uninstalled matplotlib-3.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
plotnine 0.10.1 requires matplotlib>=3.5.0, but you have matplotlib 3.1.3 which is incompatible.
pandas-profiling 3.2.0 requires 

In [10]:
PROJECT_PATH = '/content/'

In [11]:
%cd {PROJECT_PATH}

/content


In [12]:
!ls

sample_data


In [13]:
!git clone https://github.com/openvinotoolkit/anomalib.git

Cloning into 'anomalib'...
remote: Enumerating objects: 27705, done.
remote: Counting objects: 100% (920/920), done.
remote: Compressing objects: 100% (644/644), done.
remote: Total 27705 (delta 295), reused 838 (delta 258), pack-reused 26785
Receiving objects: 100% (27705/27705), 1.50 GiB | 27.03 MiB/s, done.
Resolving deltas: 100% (15240/15240), done.


In [14]:
%cd anomalib

/content/anomalib


In [15]:
!ls

CHANGELOG.md	    LICENSE	    requirements	      tools
CITATION.cff	    MANIFEST.in     setup.py		      tox.ini
CODE_OF_CONDUCT.md  notebooks	    src
CONTRIBUTING.md     pyproject.toml  tests
docs		    README.md	    third-party-programs.txt


In [16]:
!pip install -e . -q

  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for anomalib (pyproject.toml) ... done


In [18]:
import anomalib


In [19]:
%cd ..

/content


In [20]:
!ls

anomalib  sample_data


In [21]:
!curl --help

Usage: curl [options...] <url>
     --abstract-unix-socket <path> Connect via abstract Unix domain socket
     --alt-svc <file name> Enable alt-svc with this cache file
     --anyauth       Pick any authentication method
 -a, --append        Append to target file when uploading
     --basic         Use HTTP Basic Authentication
     --cacert <file> CA certificate to verify peer against
     --capath <dir>  CA directory to verify peer against
 -E, --cert <certificate[:password]> Client certificate file and password
     --cert-status   Verify the status of the server certificate
     --cert-type <type> Certificate file type (DER/PEM/ENG)
     --ciphers <list of ciphers> SSL ciphers to use
     --compressed    Request compressed response
     --compressed-ssh Enable SSH compression
 -K, --config <file> Read config from a file
     --connect-timeout <seconds> Maximum time allowed for connection
     --connect-to <HOST1:PORT1:HOST2:PORT2> Connect to host
 -C, --continue-at <offset> Resumed

In [22]:
! curl https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937637-1629952063/metal_nut.tar.xz --output metal_nut.tar.xz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  157M  100  157M    0     0  12.7M      0  0:00:12  0:00:12 --:--:-- 15.3M


In [23]:
!tar xf metal_nut.tar.xz

## Importing libraries

In [24]:
import numpy as np
import matplotlib.pyplot as plt
import os, pprint, yaml, warnings, math, glob, cv2, random, logging
from IPython.display import Image

In [25]:
def warn(*args, **kwargs):
    pass
warnings.warn = warn
warnings.filterwarnings('ignore')
logger = logging.getLogger("anomalib")

In [26]:
import anomalib
from pytorch_lightning import Trainer, seed_everything
from anomalib.config import get_configurable_parameters
from anomalib.data import get_datamodule
from anomalib.models import get_model
from anomalib.utils.callbacks import LoadModelCallback, get_callbacks
from anomalib.utils.loggers import configure_logger, get_experiment_logger

ModuleNotFoundError: ignored

In [ ]:
import torch
print("Torch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
print("GPU availability:", torch.cuda.is_available())
print("Number of GPU devices:", torch.cuda.device_count())      
print("Name of current GPU:", torch.cuda.get_device_name(0))  

## Checking the dataset

In [ ]:
def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)

In [ ]:
#%cd {PROJECT_PATH}/dataset/MVTec/

In [ ]:
!ls

In [ ]:
list_files("metal_nut")

## Models

### PaDiM: A Patch Distribution Modeling Framework for Anomaly Detection and Localization

Paper: [PaDiM](https://arxiv.org/pdf/2011.08785.pdf)

PaDiM is a patch based algorithm. It relies on a pre-trained CNN feature extractor. The image is broken into patches and embeddings are extracted from each patch using different layers of the feature extractors. The activation vectors from different layers are concatenated to get embedding vectors carrying information from different semantic levels and resolutions. This helps encode fine grained and global contexts. However, since the generated embedding vectors may carry redundant information, dimensions are reduced using random selection. A multivariate gaussian distribution is generated for each patch embedding across the entire training batch. Thus, for each patch of the set of training images, we have a different multivariate gaussian distribution. These gaussian distributions are represented as a matrix of gaussian parameters.

During inference, Mahalanobis distance is used to score each patch position of the test image. It uses the inverse of the covariance matrix calculated for the patch during training. The matrix of Mahalanobis distances forms the anomaly map with higher scores indicating anomalous regions.

![PaDiM Architecture](https://raw.githubusercontent.com/openvinotoolkit/anomalib/development/docs/source/images/padim/architecture.jpg "PaDiM Architecture")

### PatchCore

Paper: [PatchCore](https://arxiv.org/pdf/2106.08265.pdf)

The PatchCore algorithm is based on the idea that an image can be classified as anomalous as soon as a single patch is anomalous. The input image is tiled. These tiles act as patches which are fed into the neural network. It consists of a single pre-trained network which is used to extract "mid" level features patches. The "mid" level here refers to the feature extraction layer of the neural network model. Lower level features are generally too broad and higher level features are specific to the dataset the model is trained on. The features extracted during training phase are stored in a memory bank of neighbourhood aware patch level features.

During inference this memory bank is coreset subsampled. Coreset subsampling generates a subset which best approximates the structure of the available set and allows for approximate solution finding. This subset helps reduce the search cost associated with nearest neighbour search. The anomaly score is taken as the maximum distance between the test patch in the test patch collection to each respective nearest neighbour.

![PatchCore Architecture](https://raw.githubusercontent.com/openvinotoolkit/anomalib/development/docs/source/images/patchcore/architecture.jpg "PatchCore Architecture")

### Student-Teacher Feature Pyramid Matching for Unsupervised Anomaly Detection

Paper: [STFPM](https://arxiv.org/pdf/2103.04257.pdf)

STFPM algorithm consists of a pre-trained teacher network and a student network with identical architecture. The student network learns the distribution of anomaly-free images by matching the features with the counterpart features in the teacher network. Multi-scale feature matching is used to enhance robustness. This hierarchical feature matching enables the student network to receive a mixture of multi-level knowledge from the feature pyramid thus allowing for anomaly detection of various sizes.

During inference, the feature pyramids of teacher and student networks are compared. Larger difference indicates a higher probability of anomaly occurrence.

![STFPM Architecture](https://raw.githubusercontent.com/openvinotoolkit/anomalib/development/docs/source/images/stfpm/architecture.jpg "STFPM Architecture")

### FastFlow: Unsupervised Anomaly Detection and Localization via 2D Normalizing Flows

Paper: [FastFlow](https://arxiv.org/abs/2111.07677)

FastFlow is a two-dimensional normalizing flow-based probability distribution estimator. It can be used as a plug-in module with any deep feature extractor, such as ResNet and vision transformer, for unsupervised anomaly detection and localisation. In the training phase, FastFlow learns to transform the input visual feature into a tractable distribution, and in the inference phase, it assesses the likelihood of identifying anomalies.

![FastFlow Architecture](https://raw.githubusercontent.com/openvinotoolkit/anomalib/development/docs/source/images/fastflow/architecture.jpg "FastFlow Architecture")

### Anomaly Detection via Reverse Distillation from One-Class Embedding

Paper: [Reverse Distillation](https://arxiv.org/pdf/2201.10703v2.pdf)

Reverse Distillation model consists of three networks. The first is a pre-trained feature extractor (E). The next two are the one-class bottleneck embedding (OCBE) and the student decoder network (D). The backbone E is a ResNet model pre-trained on ImageNet dataset. During the forward pass, features from three ResNet block are extracted. These features are encoded by concatenating the three feature maps using the multi-scale feature fusion block of OCBE and passed to the decoder D. The decoder network is symmetrical to the feature extractor but reversed. During training, outputs from these symmetrical blocks are forced to be similar to the corresponding feature extractor layers by using cosine distance as the loss metric.

During testing, a similar step is followed but this time the cosine distance between the feature maps is used to indicate the presence of anomalies. The distance maps from all the three layers are up-sampled to the image size and added (or multiplied) to produce the final feature map. Gaussian blur is applied to the output map to make it smoother. Finally, the anomaly map is generated by applying min-max normalization on the output map.

![Anomaly Detection via Reverse Distillation from One-Class Embedding Architecture](https://raw.githubusercontent.com/openvinotoolkit/anomalib/development/docs/source/images/reverse_distillation/architecture.png "Reverse Distillation Architecture")


## Setting config files

In [ ]:
%cd {'/content'}

In [ ]:
!ls

In [ ]:
CONFIG_PATHS = '/content' + '/anomalib/anomalib/models'
MODEL_CONFIG_PAIRS = {
    'patchcore': f'{CONFIG_PATHS}/patchcore/config.yaml',
    'padim':     f'{CONFIG_PATHS}/padim/config.yaml',
    'cflow':     f'{CONFIG_PATHS}/cflow/config.yaml',
    'dfkde':     f'{CONFIG_PATHS}/dfkde/config.yaml',
    'dfm':       f'{CONFIG_PATHS}/dfm/config.yaml',
    'ganomaly':  f'{CONFIG_PATHS}/ganomaly/config.yaml',
    'stfpm':     f'{CONFIG_PATHS}/stfpm/config.yaml',
    'fastflow':  f'{CONFIG_PATHS}/fastflow/config.yaml',
    'draem':     f'{CONFIG_PATHS}/draem/config.yaml',
    'reverse_distillation': f'{CONFIG_PATHS}/reverse_distillation/config.yaml',
}

In [ ]:
MODEL = 'reverse_distillation'
print(open(os.path.join(MODEL_CONFIG_PAIRS[MODEL]), 'r').read())

In [ ]:
new_update = {
    "path": '/content',
    'task': 'segmentation',
    'category': 'metal_nut', 
    'image_size': 256,
    'train_batch_size': 4,
    'test_batch_size': 4,
    'max_epochs': 4,
    'seed': 101
}

In [ ]:
# update yaml key's value
def update_yaml(old_yaml, new_yaml, new_update):
    # load yaml
    with open(old_yaml) as f:
        old = yaml.safe_load(f)
                  
    temp = []
    def set_state(old, key, value):
        if isinstance(old, dict):
            for k, v in old.items():
                if k == 'project':
                    temp.append(k)
                if k == key:
                    if temp and k == 'path':
                        # right now, we don't wanna change `project.path`
                        continue
                    old[k] = value
                elif isinstance(v, dict):
                    set_state(v, key, value)
    
    # iterate over the new update key-value pari
    for key, value in new_update.items():
        set_state(old, key, value)
    
    # save the updated / modified yaml file
    with open(new_yaml, 'w') as f:
        yaml.safe_dump(old, f, default_flow_style=False)

In [ ]:
# let's set a new path location of new config file 
new_yaml_path = CONFIG_PATHS + '/' + MODEL + '_new.yaml'

In [ ]:
new_yaml_path

In [ ]:
# run the update yaml method to update desired key's values
update_yaml(MODEL_CONFIG_PAIRS[MODEL], new_yaml_path, new_update)        

In [ ]:
with open(new_yaml_path) as f:
    updated_config = yaml.safe_load(f)
pprint.pprint(updated_config) # check if it's updated

##  Training

In [ ]:
if updated_config['project']['seed'] != 0:
    print(updated_config['project']['seed'])
    seed_everything(updated_config['project']['seed'])

In [ ]:
# It will return the configurable parameters in DictConfig object.
config = get_configurable_parameters(
    model_name=updated_config['model']['name'],
    config_path=new_yaml_path
)

In [ ]:
# pass the config file to model, logger, callbacks and datamodule
model      = get_model(config)
experiment_logger = get_experiment_logger(config)
callbacks  = get_callbacks(config)
datamodule = get_datamodule(config)

In [ ]:
# start training
trainer = Trainer(**config.trainer, logger=experiment_logger, callbacks=callbacks)
trainer.fit(model=model, datamodule=datamodule)

## Evaluation

In [ ]:
# load best model from checkpoint before evaluating
load_model_callback = LoadModelCallback(
    weights_path=trainer.checkpoint_callback.best_model_path
)
trainer.callbacks.insert(0, load_model_callback)
trainer.test(model=model, datamodule=datamodule)

## Visualization of the prediction on the test dataset

In [ ]:
RESULT_PATH = os.path.join(
    updated_config['project']['path'],
    updated_config['model']['name'],
    updated_config['dataset']['format'], 
    updated_config['dataset']['category']
)
RESULT_PATH

In [ ]:
# a simple function to visualize the model's prediction (anomaly heatmap)
def visualiz(paths, n_images, is_random=True, figsize=(16, 16)):
    for i in range(n_images):
        image_name = paths[i]
        if is_random: image_name = random.choice(paths)
        img = cv2.imread(image_name)[:,:,::-1]
        
        category_type = image_name.split('/')[-4:-3:][0]
        defected_type = image_name.split('/')[-2:-1:][0]
        
        plt.figure(figsize=figsize)
        plt.imshow(img)
        plt.title(
            f"Category : {category_type} and Defected Type : {defected_type} \n {image_name}", 
            fontdict={'fontsize': 20, 'fontweight': 'medium'}
        )
        plt.xticks([])
        plt.yticks([])
        plt.tight_layout()
    plt.show()

In [ ]:
for content in os.listdir(RESULT_PATH):
    if content == 'images':
        full_path = glob.glob(os.path.join(RESULT_PATH, content, '**',  '*.png'), recursive=True)
        print('Total Image ', len(full_path))
        print(full_path[0].split('/'))
        print(full_path[0].split('/')[-2:-1:])
        print(full_path[0].split('/')[-4:-3:])

In [ ]:
visualiz(full_path, 10, is_random=True, figsize=(30, 30))

## Inference on new images

In [ ]:
PROJECT_PATH = "/content"

In [ ]:
%cd {PROJECT_PATH}

In [ ]:
!ls

In [ ]:
infer_results = PROJECT_PATH + "/infer_results"
infer_results

In [ ]:
# anomalies: color, bent, flip, scratch
# images: 1 to ~20

In [ ]:
# input image
input_img = PROJECT_PATH + "/metal_nut/test/bent/013.png"
input_img

In [ ]:
# output image
output_img = infer_results +  "/metal_nut/test/bent/013.png"
output_img

In [ ]:
from anomalib.config import get_configurable_parameters
from anomalib.data.inference import InferenceDataset
from anomalib.models import get_model
from anomalib.utils.callbacks import get_callbacks

In [ ]:
from argparse import ArgumentParser, Namespace
from pathlib import Path

from pytorch_lightning import Trainer
from torch.utils.data import DataLoader

In [ ]:
trainer

In [ ]:
def infer(trainer):
    """Run inference."""

    #args = get_args()
    config = get_configurable_parameters(config_path=new_yaml_path)
    config.trainer.resume_from_checkpoint = str(trainer.checkpoint_callback.best_model_path)
    #config.visualization.show_images = args.show
    config.visualization.mode = "simple"
    if infer_results:  # overwrite save path
        config.visualization.save_images = True
        config.visualization.image_save_path = infer_results
    else:
        config.visualization.save_images = False

    model = get_model(config)
    callbacks = get_callbacks(config)

    trainer = Trainer(callbacks=callbacks, **config.trainer)

    transform_config = config.dataset.transform_config.val if "transform_config" in config.dataset.keys() else None
    dataset = InferenceDataset(
        input_img, image_size=tuple(config.dataset.image_size), transform_config=transform_config
    )
    dataloader = DataLoader(dataset)
    trainer.predict(model=model, dataloaders=[dataloader])

In [ ]:
infer(trainer)

In [ ]:
# perform inference on the sample image
#!python -W ignore anomalib/tools/inference/lightning_inference.py \
#        --config {new_yaml_path} \
#        --weights {trainer.checkpoint_callback.best_model_path} \
#        --input {input_img} \
#        --output infer_results

In [ ]:
print(input_img)
display(Image(input_img, width=250))

In [ ]:
display(Image("/content/infer_results/bent/013.png", width=250))

## Summary of evaluating some arquitectures

```
MODEL = 'padim'
```

```
'image_size': 256,
'train_batch_size': 4,
'test_batch_size': 4,
'max_epochs': 4,
```

```
[{'pixel_F1Score': 0.7553240060806274,
  'pixel_AUROC': 0.9686623215675354,
  'image_F1Score': 0.952380895614624,
  'image_AUROC': 0.9496578574180603}]
```

```
MODEL = 'patchcore'
```

```
'image_size': 128,
'train_batch_size': 1,
'test_batch_size': 1,
'max_epochs': 3,
```

```
[{'pixel_F1Score': 0.8109800219535828,
  'pixel_AUROC': 0.9827464818954468,
  'image_F1Score': 0.9726775884628296,
  'image_AUROC': 0.9833822250366211}]
```

```
MODEL = 'stfpm'
```

```
'image_size': 256,
'train_batch_size': 4,
'test_batch_size': 4,
'max_epochs': 4,
```

```
[{'pixel_F1Score': 0.6699215769767761,
  'pixel_AUROC': 0.9740051627159119,
  'image_F1Score': 0.9555555582046509,
  'image_AUROC': 0.9838709235191345}]
```

```
MODEL = 'fastflow'
```

```
'image_size': 256,
'train_batch_size': 4,
'test_batch_size': 4,
'max_epochs': 4,
```

```
[{'pixel_F1Score': 0.7417428493499756,
  'pixel_AUROC': 0.9636613130569458,
  'image_F1Score': 0.9560439586639404,
  'image_AUROC': 0.9310851097106934}]
```

```
MODEL = 'reverse_distillation'
```

```
'image_size': 256,
'train_batch_size': 4,
'test_batch_size': 4,
'max_epochs': 4,
```

```
[{'pixel_F1Score': 0.7951029539108276,
  'pixel_AUROC': 0.980124831199646,
  'image_F1Score': 0.9723756313323975,
  'image_AUROC': 0.9672531485557556}]
```